### Z-function

  
$z[i]$  is the length of the longest string that is, at the same time, a prefix of  
$s$  and a prefix of the suffix of  
$s$  starting at  
$i$ .

Note. In this article, to avoid ambiguity, we assume  
$0$ -based indexes; that is: the first character of  
$s$  has index  
$0$  and the last one has index  
$n-1$ .

The first element of Z-function,  
$z[0]$ , is generally not well defined. In this article we will assume it is zero (although it doesn't change anything in the algorithm implementation).

#### Examples
For example, here are the values of the Z-function computed for different strings:

"aaaaa" -  
$[0, 4, 3, 2, 1]$ 
"aaabaab" -  
$[0, 2, 1, 0, 2, 1, 0]$ 
"abacaba" -  
$[0, 0, 1, 0, 3, 0, 1]$ 

#### Trivial algorithm
Formal definition can be represented in the following elementary  
$O(n^2)$  implementation.

In [18]:
def z_function_trivial(s):
    n = len(s)
    z = [0] * n
    for i in range(1, n):
        while i + z[i] < n and s[z[i]] == s[i + z[i]]:
            z[i] += 1
            print('while', z[i], i)
    return z
s = 'abcababc'
s2 = "aaaabaa"
z_function_trivial(s2)

while 1 1
while 2 1
while 3 1
while 1 2
while 2 2
while 1 3
while 1 5
while 2 5
while 1 6


[0, 3, 2, 1, 0, 2, 1]

We just iterate through every position  
$i$  and update  
$z[i]$  for each one of them, starting from  
$z[i] = 0$  and incrementing it as long as we don't find a mismatch (and as long as we don't reach the end of the line).

Of course, this is not an efficient implementation. We will now show the construction of an efficient implementation.

#### Efficient algorithm

To obtain an efficient algorithm we will compute the values of  
$z[i]$  in turn from  
$i = 1$  to  
$n - 1$  but at the same time, when computing a new value, we'll try to make the best use possible of the previously computed values.

For the sake of brevity, let's call <b>segment matches</b> those substrings that coincide with a prefix of  
$s$ . For example, the value of the desired Z-function  
$z[i]$  is the length of the segment match starting at position  
$i$  (and that ends at position  
$i + z[i] - 1$ ).

To do this, we will keep the  
$[l, r)$  indices of the rightmost segment match. That is, among all detected segments we will keep the one that ends rightmost. In a way, the index  
$r$  can be seen as the "boundary" to which our string  
$s$  has been scanned by the algorithm; everything beyond that point is not yet known.

Then, if the current index (for which we have to compute the next value of the Z-function) is  
$i$ , we have one of two options:

 
$i \geq r$  -- the current position is outside of what we have already processed.

We will then compute  
$z[i]$  with the trivial algorithm (that is, just comparing values one by one). Note that in the end, if  
$z[i] > 0$ , we'll have to update the indices of the rightmost segment, because it's guaranteed that the new  
$r = i + z[i]$  is better than the previous  
$r$ .

 
$i < r$  -- the current position is inside the current segment match  
$[l, r)$ .

Then we can use the already calculated Z-values to "initialize" the value of  
$z[i]$  to something (it sure is better than "starting from zero"), maybe even some big number.

For this, we observe that the substrings  
$s[l \dots r)$  and  
$s[0 \dots r-l)$  match. This means that as an initial approximation for  
$z[i]$  we can take the value already computed for the corresponding segment  
$s[0 \dots r-l)$ , and that is  
$z[i-l]$ .

However, the value  
$z[i-l]$  could be too large: when applied to position  
$i$  it could exceed the index  
$r$ . This is not allowed because we know nothing about the characters to the right of  
$r$ : they may differ from those required.

Here is an example of a similar scenario:

 
$$ s = "aaaabaa" $$ 
When we get to the last position ( 
$i = 6$ ), the current match segment will be  
$[5, 7)$ . Position  
$6$  will then match position  
$6 - 5 = 1$ , for which the value of the Z-function is  
$z[1] = 3$ . Obviously, we cannot initialize  
$z[6]$  to  
$3$ , it would be completely incorrect. The maximum value we could initialize it to is  
$1$  -- because it's the largest value that doesn't bring us beyond the index  
$r$  of the match segment  
$[l, r)$ .

Thus, as an initial approximation for  
$z[i]$  we can safely take:

 
$$ z_0[i] = \min(r - i,\; z[i-l]) $$ 
After having  
$z[i]$  initialized to  
$z_0[i]$ , we try to increment  
$z[i]$  by running the trivial algorithm -- because in general, after the border  
$r$ , we cannot know if the segment will continue to match or not.

Thus, the whole algorithm is split in two cases, which differ only in the initial value of  
$z[i]$ : in the first case it's assumed to be zero, in the second case it is determined by the previously computed values (using the above formula). After that, both branches of this algorithm can be reduced to the implementation of the trivial algorithm, which starts immediately after we specify the initial value.

The algorithm turns out to be very simple. Despite the fact that on each iteration the trivial algorithm is run, we have made significant progress, having an algorithm that runs in linear time. Later on we will prove that the running time is linear.

In [16]:
def z_func(s):
    n = len(s)
    z, l, r = [0] * n, 0, 0
    for i in range(1, n):
        if i < r:
            z[i] = min(r - i, z[i - l])
            print('i < r', z[i])
        while i + z[i] < n and s[z[i]] == s[i + z[i]]:
            z[i] += 1
            print('while', z[i])
        if i + z[i] > r:
            l, r = i, i + z[i]
            print('l, r', l, r)
    return z
s = 'abcababc'
s2 = "aaaabaa"
z_func(s2)

while 1
while 2
while 3
l, r 1 4
i < r 2
i < r 1
while 1
while 2
l, r 5 7
i < r 1


[0, 3, 2, 1, 0, 2, 1]

#### Comments on this implementation
The whole solution is given as a function which returns an array of length  
$n$  -- the Z-function of  
$s$ .

Array  
$z$  is initially filled with zeros. The current rightmost match segment is assumed to be  
$[0; 0)$  (that is, a deliberately small segment which doesn't contain any  
$i$ ).

Inside the loop for  
$i = 1 \dots n - 1$  we first determine the initial value  
$z[i]$  -- it will either remain zero or be computed using the above formula.

Thereafter, the trivial algorithm attempts to increase the value of  
$z[i]$  as much as possible.

In the end, if it's required (that is, if  
$i + z[i] > r$ ), we update the rightmost match segment  
$[l, r)$ .